In [ ]:
import time
import requests
from bs4 import BeautifulSoup
import feedparser
import json
import os


# 초기 설정
headers = {'User-Agent': 'Mozilla/5.0'}
session = requests.Session()
rss_url = "https://u.today/rss"
header_list_path = 'news_headers.json'

def load_history_headers():
    """과거 뉴스 헤더 리스트를 파일에서 로드합니다."""
    if os.path.exists(header_list_path):
        with open(header_list_path, 'r') as file:
            return json.load(file)
    return []

def save_history_headers(headers_list):
    """업데이트된 뉴스 헤더 리스트를 파일에 저장합니다."""
    with open(header_list_path, 'w') as file:
        json.dump(headers_list, file)

def fetch_news():
    """RSS 피드에서 새로운 뉴스를 가져와 본문을 리스트로 반환합니다."""
    new_articles = []  # 새로운 기사의 텍스트를 저장할 리스트
    history_news_header = load_history_headers()
    
    feed = feedparser.parse(rss_url)
    for entry in feed.entries:
        title = entry.title
        
        if title not in history_news_header:
            response = session.get(entry.link, headers=headers)
            soup = BeautifulSoup(response.content, 'html.parser')
            article_content = soup.select_one('div.article__content')
            if article_content:
                unwanted_section = article_content.select_one('div.article__contents')
                if unwanted_section:
                    unwanted_section.decompose()  # unwanted_section을 DOM에서 제거합니다.
                text = article_content.get_text(strip=True).replace('Advertisement','')  # 'Advertisement' 제거
                new_articles.append(text)  # 수정된 텍스트를 리스트에 추가
                history_news_header.append(title)
    
    save_history_headers(history_news_header)
    return new_articles 



def run_fetcher():
    """60초마다 새로운 뉴스의 본문을 가져오는 함수를 실행합니다."""
    while True:
        new_articles = fetch_news()
        if new_articles:
            print(f"새로운 뉴스 본문 수집됨: {new_articles}")
        else:
            print("새로운 뉴스 없음.")
        time.sleep(60)  # 60초 동안 대기

# 함수 실행
run_fetcher()